### Pt. 4 
##### Neural Network pipeline for Big G's automotive, with the following improvements:
1. Adding the 1569 partial derate
2. Because we're adjusting across multiple onehot columns, we will get errors if we reuse old code. Need something different...

In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, accuracy_score, confusion_matrix, 
    f1_score, fbeta_score, 
    matthews_corrcoef, brier_score_loss
)
from sklearn.calibration import CalibrationDisplay

from imblearn.over_sampling import RandomOverSampler

In [2]:
#read df in
#For some reason columns (7,12,15,16,18,19,20,21,22,23,25,27,28) have mixed types. Same warning as in pt. 2. to see warning, delete low_memory=False
big_g_df=pd.read_csv('../data/big_g_pipeline_ready.csv')

In [3]:
big_g_df=big_g_df.drop(columns=['Unnamed: 0',
                      ])

In [4]:
big_g_df.head()

,EventTimeStamp_x,1761_1_x,1761_10_x,1761_11_x,1761_17_x,1761_18_x,1761_19_x,1761_3_x,1761_4_x,1761_9_x,...,5862_3_y,5862_4_y,6773_16_y,6780_3_y,6802_31_y,7321_4_y,7323_4_y,7854_2_y,7854_3_y,7854_4_y
0,2015-02-21 10:47:13,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-02-21 11:40:22,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-02-21 11:40:52,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-02-21 11:42:19,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-02-21 11:14:38,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
pd.set_option('display.max_columns', None)

In [6]:
big_g_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100377 entries, 0 to 100376
Data columns (total 423 columns):
 #    Column            Dtype  
---   ------            -----  
 0    EventTimeStamp_x  object 
 1    1761_1_x          bool   
 2    1761_10_x         bool   
 3    1761_11_x         bool   
 4    1761_17_x         bool   
 5    1761_18_x         bool   
 6    1761_19_x         bool   
 7    1761_3_x          bool   
 8    1761_4_x          bool   
 9    1761_9_x          bool   
 10   3031_18_x         bool   
 11   3031_2_x          bool   
 12   3031_3_x          bool   
 13   3031_4_x          bool   
 14   3031_9_x          bool   
 15   3216_10_x         bool   
 16   3216_11_x         bool   
 17   3216_16_x         bool   
 18   3216_2_x          bool   
 19   3216_20_x         bool   
 20   3216_21_x         bool   
 21   3216_3_x          bool   
 22   3216_4_x          bool   
 23   3216_9_x          bool   
 24   3217_2_x          bool   
 25   3218_2_x          

In [7]:
# Columns (14,17,18,20,21,22,23,24,25,27,29,30) have mixed types
#updating speed, enginetimeltd and distanceltd to use
#big_g_df['BarometricPressure']=big_g_df['BarometricPressure'].str.extract('(\d+)')
#big_g_df['EngineCoolantTemperature']=big_g_df['EngineCoolantTemperature'].str.extract('(\d+)')
#big_g_df['EngineOilPressure']=big_g_df['EngineOilPressure'].str.extract('(\d+)')
#big_g_df['FuelRate']=big_g_df['FuelRate'].str.extract('(\d+)')
#big_g_df['Speed']=big_g_df['Speed'].str.extract('(\d+)')
#big_g_df['EngineTimeLtd']=big_g_df['EngineTimeLtd'].str.extract('(\d+)')
#big_g_df['DistanceLtd']=big_g_df['DistanceLtd'].str.extract('(\d+)')

In [8]:
#ask michael about this one...
#big_g_df[big_g_df[['BarometricPressure', 
#          'EngineCoolantTemperature', 
#          'EngineOilPressure', 
#          'FuelRate', 
#          'Speed', 
#          'EngineTimeLtd', 
#          'DistanceLtd']].isnull()]

In [9]:
#big_g_df[['BarometricPressure', 
#          'EngineCoolantTemperature', 
#          'EngineOilPressure', 
#          'FuelRate', 
#          'Speed', 
#          'EngineTimeLtd', 
#          'DistanceLtd']].astype(float)

In [10]:
big_g_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100377 entries, 0 to 100376
Data columns (total 423 columns):
 #    Column            Dtype  
---   ------            -----  
 0    EventTimeStamp_x  object 
 1    1761_1_x          bool   
 2    1761_10_x         bool   
 3    1761_11_x         bool   
 4    1761_17_x         bool   
 5    1761_18_x         bool   
 6    1761_19_x         bool   
 7    1761_3_x          bool   
 8    1761_4_x          bool   
 9    1761_9_x          bool   
 10   3031_18_x         bool   
 11   3031_2_x          bool   
 12   3031_3_x          bool   
 13   3031_4_x          bool   
 14   3031_9_x          bool   
 15   3216_10_x         bool   
 16   3216_11_x         bool   
 17   3216_16_x         bool   
 18   3216_2_x          bool   
 19   3216_20_x         bool   
 20   3216_21_x         bool   
 21   3216_3_x          bool   
 22   3216_4_x          bool   
 23   3216_9_x          bool   
 24   3217_2_x          bool   
 25   3218_2_x          

In [11]:
print(list(big_g_df.columns))

['EventTimeStamp_x', '1761_1_x', '1761_10_x', '1761_11_x', '1761_17_x', '1761_18_x', '1761_19_x', '1761_3_x', '1761_4_x', '1761_9_x', '3031_18_x', '3031_2_x', '3031_3_x', '3031_4_x', '3031_9_x', '3216_10_x', '3216_11_x', '3216_16_x', '3216_2_x', '3216_20_x', '3216_21_x', '3216_3_x', '3216_4_x', '3216_9_x', '3217_2_x', '3218_2_x', '3222_5_x', '3226_10_x', '3226_11_x', '3226_16_x', '3226_2_x', '3226_20_x', '3226_21_x', '3226_4_x', '3226_9_x', '3227_10_x', '3227_21_x', '3228_2_x', '3242_0_x', '3242_15_x', '3242_16_x', '3242_3_x', '3242_4_x', '3246_0_x', '3246_15_x', '3246_16_x', '3246_2_x', '3246_3_x', '3246_4_x', '3251_0_x', '3251_10_x', '3251_15_x', '3251_16_x', '3251_2_x', '3251_3_x', '3251_4_x', '3360_11_x', '3360_12_x', '3360_19_x', '3360_2_x', '3360_9_x', '3361_12_x', '3361_2_x', '3361_3_x', '3361_4_x', '3361_5_x', '3362_31_x', '3362_7_x', '3363_16_x', '3363_3_x', '3363_4_x', '3363_5_x', '3363_7_x', '3364_10_x', '3364_11_x', '3364_18_x', '3364_3_x', '3364_9_x', '3480_17_x', '3480_2_

In [12]:
features = [ '1761_1_x', '1761_10_x', '1761_11_x', '1761_17_x', '1761_18_x', '1761_19_x', '1761_3_x', '1761_4_x', 
            '1761_9_x', '3031_18_x', '3031_2_x', '3031_3_x', '3031_4_x', '3031_9_x', '3216_10_x', '3216_11_x', '3216_16_x', '3216_2_x', 
            '3216_20_x', '3216_21_x', '3216_3_x', '3216_4_x', '3216_9_x', '3217_2_x', '3218_2_x', '3222_5_x', '3226_10_x', '3226_11_x', 
            '3226_16_x', '3226_2_x', '3226_20_x', '3226_21_x', '3226_4_x', '3226_9_x', '3227_10_x', '3227_21_x', '3228_2_x', '3242_0_x', 
            '3242_15_x', '3242_16_x', '3242_3_x', '3242_4_x', '3246_0_x', '3246_15_x', '3246_16_x', '3246_2_x', '3246_3_x', '3246_4_x', 
            '3251_0_x', '3251_10_x', '3251_15_x', '3251_16_x', '3251_2_x', '3251_3_x', '3251_4_x', '3360_11_x', '3360_12_x', '3360_19_x', 
            '3360_2_x', '3360_9_x', '3361_12_x', '3361_2_x', '3361_3_x', '3361_4_x', '3361_5_x', '3362_31_x', '3362_7_x', '3363_16_x', 
            '3363_3_x', '3363_4_x', '3363_5_x', '3363_7_x', '3364_10_x', '3364_11_x', '3364_18_x', '3364_3_x', '3364_9_x', '3480_17_x', 
            '3480_2_x', '3480_3_x', '3480_4_x', '3482_2_x', '3482_3_x', '3482_7_x', '3490_3_x', '3490_4_x', '3490_7_x', '3515_10_x', 
            '3521_18_x', '3556_18_x', '3556_2_x', '3556_5_x', '3610_2_x', '3610_3_x', '3610_4_x', '3703_31_x', '3720_15_x', '3936_14_x',
            '3936_15_x', '3936_16_x', '3936_7_x', '4094_18_x', '4094_31_x', '4096_31_x', '4331_16_x', '4331_18_x', '4334_16_x', '4334_18_x', 
            '4334_2_x', '4334_3_x', '4334_4_x', '4339_7_x', '4340_3_x', '4340_4_x', '4340_5_x', '4342_3_x', '4342_4_x', '4342_5_x', '4344_3_x', 
            '4344_4_x', '4344_5_x', '4346_5_x', '4360_0_x', '4360_10_x', '4360_16_x', '4360_3_x', '4360_4_x', '4363_0_x', '4363_10_x', '4363_16_x',
            '4363_2_x', '4363_3_x', '4363_4_x', '4364_18_x', '4364_31_x', '4375_2_x', '4375_4_x', '4376_3_x', '4376_4_x', '4376_5_x', '4376_7_x', 
            '4765_16_x', '4765_2_x', '4765_3_x', '4765_4_x', '4766_15_x', '4766_3_x', '4792_14_x', '4794_31_x', '4795_31_x', '4796_31_x', '5024_10_x',
            '5031_10_x', '520953_4_x', '521032_14_x', '5246_16_x', '5298_17_x', '5298_18_x', '5319_31_x', '5392_31_x', '5394_3_x',
            '5394_4_x', '5394_5_x', '5394_7_x', '5397_31_x', '5491_3_x', '5491_4_x', '5491_5_x', '5491_7_x', '5569_2_x', '5742_11_x', '5742_12_x', 
            '5742_16_x', '5742_3_x', '5742_4_x', '5742_9_x', '5743_11_x', '5743_12_x', '5743_3_x', '5743_4_x', '5743_9_x', '5745_18_x', '5745_3_x', 
            '5745_4_x', '5746_4_x', '5835_21_x', '5835_3_x', '5835_4_x', '5835_9_x', '5848_12_x', '5848_13_x', '5848_19_x', '5848_4_x', '5848_9_x', 
            '5851_18_x', '5851_2_x', '5853_10_x', '5862_0_x', '5862_16_x', '5862_2_x', '5862_3_x', '5862_4_x', '6773_16_x', '6780_3_x', '6802_31_x', 
            '7321_4_x', '7323_4_x', '7854_2_x', '7854_3_x', '7854_4_x', '1761_1_y', '1761_10_y', '1761_11_y', '1761_17_y', '1761_18_y', 
            '1761_19_y', '1761_3_y', '1761_4_y', '1761_9_y', '3031_18_y', '3031_2_y', '3031_3_y', '3031_4_y', '3031_9_y', '3216_10_y', '3216_11_y',
            '3216_16_y', '3216_2_y', '3216_20_y', '3216_21_y', '3216_3_y', '3216_4_y', '3216_9_y', '3217_2_y', '3218_2_y', '3222_5_y', '3226_10_y', 
            '3226_11_y', '3226_16_y', '3226_2_y', '3226_20_y', '3226_21_y', '3226_4_y', '3226_9_y', '3227_10_y', '3227_21_y', '3228_2_y', '3242_0_y', 
            '3242_15_y', '3242_16_y', '3242_3_y', '3242_4_y', '3246_0_y', '3246_15_y', '3246_16_y', '3246_2_y', '3246_3_y', '3246_4_y', '3251_0_y', 
            '3251_10_y', '3251_15_y', '3251_16_y', '3251_2_y', '3251_3_y', '3251_4_y', '3360_11_y', '3360_12_y', '3360_19_y', '3360_2_y', '3360_9_y',
            '3361_12_y', '3361_2_y', '3361_3_y', '3361_4_y', '3361_5_y', '3362_31_y', '3362_7_y', '3363_16_y', '3363_3_y', '3363_4_y', '3363_5_y',
            '3363_7_y', '3364_10_y', '3364_11_y', '3364_18_y', '3364_3_y', '3364_9_y', '3480_17_y', '3480_2_y', '3480_3_y', '3480_4_y', '3482_2_y', 
            '3482_3_y', '3482_7_y', '3490_3_y', '3490_4_y', '3490_7_y', '3515_10_y', '3521_18_y', '3556_18_y', '3556_2_y', '3556_5_y', '3610_2_y',
            '3610_3_y', '3610_4_y', '3703_31_y', '3720_15_y', '3936_14_y', '3936_15_y', '3936_16_y', '3936_7_y', '4094_18_y', '4094_31_y', '4096_31_y',
            '4331_16_y', '4331_18_y', '4334_16_y', '4334_18_y', '4334_2_y', '4334_3_y', '4334_4_y', '4339_7_y', '4340_3_y', '4340_4_y', '4340_5_y', 
            '4342_3_y', '4342_4_y', '4342_5_y', '4344_3_y', '4344_4_y', '4344_5_y', '4346_5_y', '4360_0_y', '4360_10_y', '4360_16_y', '4360_3_y', 
            '4360_4_y', '4363_0_y', '4363_10_y', '4363_16_y', '4363_2_y', '4363_3_y', '4363_4_y', '4364_18_y', '4364_31_y', '4375_2_y', '4375_4_y', 
            '4376_3_y', '4376_4_y', '4376_5_y', '4376_7_y', '4765_16_y', '4765_2_y', '4765_3_y', '4765_4_y', '4766_15_y', '4766_3_y', '4792_14_y', 
            '4794_31_y', '4795_31_y', '4796_31_y', '5024_10_y', '5031_10_y', '520953_4_y', '521032_14_y', '5246_16_y', '5298_17_y', '5298_18_y', 
            '5319_31_y', '5392_31_y', '5394_3_y', '5394_4_y', '5394_5_y', '5394_7_y', '5397_31_y', '5491_3_y', '5491_4_y', '5491_5_y', '5491_7_y', 
            '5569_2_y', '5742_11_y', '5742_12_y', '5742_16_y', '5742_3_y', '5742_4_y', '5742_9_y', '5743_11_y', '5743_12_y', '5743_3_y', '5743_4_y',
            '5743_9_y', '5745_18_y', '5745_3_y', '5745_4_y', '5746_4_y', '5835_21_y', '5835_3_y', '5835_4_y', '5835_9_y', '5848_12_y', '5848_13_y', 
            '5848_19_y', '5848_4_y', '5848_9_y', '5851_18_y', '5851_2_y', '5853_10_y', '5862_0_y', '5862_16_y', '5862_2_y', '5862_3_y', '5862_4_y',
            '6773_16_y', '6780_3_y', '6802_31_y', '7321_4_y', '7323_4_y', '7854_2_y', '7854_3_y', '7854_4_y']

#Searching for both full and partial derate
y = big_g_df[['5246_0', '1569_31']]

# Features selection has a few added...
X = big_g_df[features]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=321)

# Scaling
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
from keras.layers import Dense

#Start with a sequential model
model = tf.keras.Sequential()

# Then add Dense layer
# See https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
model.add(
    tf.keras.layers.Dense(units = 2,
                          input_shape = (len(features),),
                          activation = 'tanh'
                          )
)

# And end with another Dense layer as the output layer.
# It needs one node per target category
# may want to use something different than softmax, per ds article...
# https://en.wikipedia.org/wiki/Softmax_function
model.add(tf.keras.layers.Dense(2, activation = 'relu'))
model.add(tf.keras.layers.Dense(2, activation = 'relu'))
model.add(tf.keras.layers.Dense(2, activation = 'relu'))
model.add(Dense(2, activation='sigmoid'))

# Finally, compile the model
# We need to use sparse categorical crossentropy since our target is encoded as integers
# We can also give one or more metrics we want to track as we train out model.


In [27]:
#from keras.models import Sequential
#from keras.layers import Dense

#From the Jason Brownlee book, I think each separate selection can be tried w/ a 
#sigmoid activation, because they have different distributions???
#model = Sequential()
#model.add(Dense(64, input_dim=len(features), activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(2, activation='sigmoid'))  # Output layer for two binary labels



In [28]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:

# Fit model on scaled data
model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=15, batch_size=32)

Epoch 1/15
2353/2353 [==============================] - 12s 5ms/step - loss: nan - accuracy: 0.2402 - val_loss: nan - val_accuracy: 0.9944
Epoch 2/15
2353/2353 [==============================] - 9s 4ms/step - loss: nan - accuracy: 0.9944 - val_loss: nan - val_accuracy: 0.9944
Epoch 3/15
2353/2353 [==============================] - 14s 6ms/step - loss: nan - accuracy: 0.9944 - val_loss: nan - val_accuracy: 0.9944
Epoch 4/15
2353/2353 [==============================] - 10s 4ms/step - loss: nan - accuracy: 0.9944 - val_loss: nan - val_accuracy: 0.9944
Epoch 5/15
2353/2353 [==============================] - 9s 4ms/step - loss: nan - accuracy: 0.9944 - val_loss: nan - val_accuracy: 0.9944
Epoch 6/15
2353/2353 [==============================] - 9s 4ms/step - loss: nan - accuracy: 0.9944 - val_loss: nan - val_accuracy: 0.9944
Epoch 7/15
2353/2353 [==============================] - 10s 4ms/step - loss: nan - accuracy: 0.9944 - val_loss: nan - val_accuracy: 0.9944
Epoch 8/15
2353/2353 [========

In [18]:
model.predict(X_test_scaled)

785/785 [==============================] - 2s 3ms/step


array([[nan, nan],
       [nan, nan],
       [nan, nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]], dtype=float32)

In [20]:
y_pred = np.argmax(model.predict(X_test_scaled), axis = 1)
y_pred

785/785 [==============================] - 2s 2ms/step


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [31]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets